In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['WANDB_MODE'] = 'dryrun'

In [3]:
import wandb
wandb.init(project="sky-eye")
conf = wandb.config

In [4]:
from glob import glob
train_dir = '/home/jupyter/datasets/xview/train'
test_dir = '/home/jupyter/datasets/xview/test'

In [5]:
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torch
from torch import nn

In [6]:
conf.aug_prob = .5

In [7]:
import albumentations as al

augment = al.Compose([
        al.HorizontalFlip(p=conf.aug_prob),
        al.VerticalFlip(p=conf.aug_prob),
        al.RandomRotate90(p=conf.aug_prob),
        al.Transpose(p=conf.aug_prob),
        al.GridDistortion(p=conf.aug_prob, distort_limit=.2),
        al.ShiftScaleRotate(p=conf.aug_prob),
        al.RandomBrightnessContrast(p=conf.aug_prob)
])

In [8]:
conf.batch_size = 4

In [9]:
from xv.nn import dataset
from xv import util

train_dataset = dataset.XViewSegmentationDataset(path=train_dir, resolution=(1024,1024), augment=augment)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=conf.batch_size,
    shuffle=True,
    num_workers=10,
    pin_memory=True,
)

In [23]:
from collections import Counter
counts = Counter(len(i['pre']['features']) for i in train_dataset.instances)

In [26]:
counts[0]/sum(counts.values())

0.18435155412647375

In [10]:
from xv.nn.losses import loss_dict, WeightedLoss

conf.loss_weights = {
    'dice': 1,
    'focal': 1
}

loss = WeightedLoss({loss_dict[l]():w for l, w in conf.loss_weights.items()})

In [11]:
conf.epochs = 100
conf.pretrained_model = "xdxd_spacenet4"
#conf.pretrained_model = 'selimsef_spacenet4_densenet121unet'

# xdxd_spacenet4

In [12]:
from xv.nn.solaris.model_io import get_model
model = get_model(conf.pretrained_model, 'torch').to('cuda')

In [13]:
from torch import optim

optims = {
    'adam': optim.Adam,
    'adamw': optim.AdamW
}

conf.optim = 'adam'
conf.lr = 0.001

optim = optims[conf.optim](model.parameters(), lr=conf.lr)

In [14]:
from apex import amp
conf.amp_opt_level = 'O2'
model, optim = amp.initialize(model, optim, opt_level=conf.amp_opt_level)

Selected optimization level O2:  FP16 training with FP32 batchnorm and FP32 master weights.

Defaults for this optimization level are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic


In [15]:
from torchvision.models.resnet import BasicBlock, Bottleneck

In [16]:
def train(model, optim, data, loss_fn):
    model.train()
    for batch in tqdm(iter(data)):
        loss_buildings = loss_fn(model(batch['images']['image'].to('cuda')),
                                 batch['masks']['buildings'].to('cuda'))

        optim.zero_grad()
        
        with amp.scale_loss(loss_buildings, optim) as scaled_loss:
            scaled_loss.backward()
            
        loss_damage = loss_fn(model(batch['images']['post'].to('cuda')),
                           batch['masks']['damage'])
        
        with amp.scale_loss(loss_damage, optim) as scaled_loss:
            scaled_loss.backward()
        
        optim.zero_grad()

In [ ]:
# optimisation to find good building detector – polygons based on it

In [17]:
train(model, optim, train_loader, loss)

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2479: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


RuntimeError: The size of tensor a (1048576) must match the size of tensor b (4194304) at non-singleton dimension 1

In [ ]:
from torchvision.models.resnet import BasicBlock

In [ ]:
model.num_channels

In [ ]:
class BuildingDamageHeatmap(nn.Module):
    def __init__(self, segmentation):
        self.segmentation = segmentation
        

In [ ]:
ix = 1000
i = train_dataset[ix]
images, masks = i['images'], i['masks']
image = images['post']
image = np.array(train_dataset.inverse_transform_image(image))

util.vis_im_mask(image, masks['damage'], size=(512*2,512*2), opacity=.3);